In [1]:
import algorithm as al
import numpy as np
import cv2 as cv
import Queue
from patchdb import *
from psi import *
from copyutils import *
from debug import *
import inspect

[INFO              ] [Logger      ] Record log in C:\Users\xiaoy\.kivy\logs\kivy_16-04-08_13.txt
[INFO              ] [Kivy        ] v1.9.1
[INFO              ] [Python      ] v2.7.11 |Continuum Analytics, Inc.| (default, Feb 16 2016, 09:58:36) [MSC v.1500 64 bit (AMD64)]
[INFO              ] [Image       ] Providers: img_tex, img_dds, img_gif, img_sdl2 (img_pil, img_ffpyplayer ignored)


In [2]:
def imshow(img):
    cv.imshow('image', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
_p = al.Inpainting()

In [4]:
_p.readImage('../../test_images/Kanizsa-triangle-tiny.png', 'source')

_p.readImage('../../test_images/Kanizsa-triangle-mask-tiny.png', 'alpha')

(True, 'Success')

In [5]:
_p._images['filled'] = np.uint8(_p._images['alpha'] > 0)*255
_p._images['inpainted'] = _p._images['source'].copy()

In [6]:
for i in range(0,3):
    _p._images['inpainted'][:,:,i] *= (_p._images['filled']>0)

#
# Step 1a,b: Identify the fill front deltaOmega and compute
#            initial patch priorities

In [7]:
_p.computeBoundaries()
_p.confidenceInitialize()

In [8]:
_p._w = 5

In [9]:
_p._patchDB = PatchDB(_p._images['inpainted'], _p._w, filled=_p._images['filled'])

In [10]:
_p.iterationsInit()
done = False

In [11]:
boundary = _p._boundaryIterator.next()
_p.fillFrontInitialize(boundary, imviewer=None)

In [12]:
boundaryPixels = boundary
# initialize the pixels on the fill front
_p._images['fillFront'] = \
    np.zeros_like(_p._images['filled'], dtype=np.uint8)
_p._images['fillFront'] = \
    cv.drawContours(_p._images['fillFront'],boundaryPixels,-1,255)

# initialize the priority queue with all points on the fill front
_p._deltaOmega = Queue.PriorityQueue()

In [52]:
# col, row = boundaryPixels[10][0]
col, row = (69, 128)
p = PSI((row, col), _p._w, 
          image=_p._images['inpainted'], 
          filled=_p._images['filled'],
          confidence=_p._images['confidence'],
          fillFront=_p._images['fillFront'])
inpaintedImage=_p._images['inpainted'] 
filledImage=_p._images['filled']
confidence=_p._images['confidence']
fillFront=_p._images['fillFront']

In [224]:
w = 3
inpainted, _ = copyutils.getWindow(inpaintedImage, p._coords, w)
filled, _ = copyutils.getWindow(filledImage, p._coords, w)
if len(inpainted.shape) == 3:
    inpainted = cv.cvtColor(inpainted, cv.COLOR_BGR2GRAY)
# Sobel filter
size = 2 * w + 1
Dx = cv.Sobel(src=inpainted, ddepth=cv.CV_32F, dx=0, dy=1, ksize=size)
Dy = -cv.Sobel(src=inpainted, ddepth=cv.CV_32F, dx=1, dy=0, ksize=size)
Dx *= filled>0
Dy *= filled>0
d = np.sqrt(Dx**2 + Dy**2)
dmax = d == d.max()
# Dx = Dx[dmax][0]
# Dy = Dy[dmax][0]

In [225]:
Dx

array([[     0.,      0.,      0.,      0.,      0.,      0.,      0.],
       [ 28050.,  38250.,      0.,      0.,      0.,      0.,      0.],
       [ 33150.,  48450.,  70125.,      0.,      0.,      0.,      0.],
       [ 20400.,  35955.,  65280.,      0.,      0.,      0.,      0.],
       [  8160.,  20400.,  49470.,  70125.,      0.,      0.,      0.],
       [  2550.,   8925.,  26775.,  44625.,      0.,      0.,      0.],
       [     0.,      0.,      0.,      0.,      0.,      0.,      0.]], dtype=float32)

In [226]:
Dy

array([[     -0.,  132600.,       0.,       0.,       0.,       0.,
             -0.],
       [     -0.,  122400.,       0.,       0.,       0.,       0.,
             -0.],
       [     -0.,   96900.,  154275.,       0.,       0.,       0.,
             -0.],
       [     -0.,   66045.,  134130.,       0.,       0.,       0.,
             -0.],
       [     -0.,   38250.,  103530.,  154275.,       0.,       0.,
             -0.],
       [     -0.,   19635.,   75225.,  144075.,       0.,       0.,
             -0.],
       [     -0.,   13260.,   63750.,  137700.,  160650.,       0.,
             -0.]], dtype=float32)

In [227]:
filled

array([[255, 255,   0,   0,   0,   0,   0],
       [255, 255,   0,   0,   0,   0,   0],
       [255, 255, 255,   0,   0,   0,   0],
       [255, 255, 255,   0,   0,   0,   0],
       [255, 255, 255, 255,   0,   0,   0],
       [255, 255, 255, 255,   0,   0,   0],
       [255, 255, 255, 255, 255,   0,   0]], dtype=uint8)

In [216]:
Dx * (filled>0)

array([[   0.,    0.,    0.],
       [ 510.,    0.,    0.],
       [   0.,    0.,    0.]], dtype=float32)

In [217]:
Dy * (filled>0)

array([[   -0.,     0.,    -0.],
       [   -0.,     0.,    -0.],
       [   -0.,  1020.,    -0.]], dtype=float32)

In [228]:
d

array([[      0.        ,  132600.        ,       0.        ,
              0.        ,       0.        ,       0.        ,       0.        ],
       [  28050.        ,  128237.3671875 ,       0.        ,
              0.        ,       0.        ,       0.        ,       0.        ],
       [  33150.        ,  108337.4921875 ,  169464.71875   ,
              0.        ,       0.        ,       0.        ,       0.        ],
       [  20400.        ,   75197.765625  ,  149172.171875  ,
              0.        ,       0.        ,       0.        ,       0.        ],
       [   8160.        ,   43350.        ,  114742.0703125 ,
         169464.71875   ,       0.        ,       0.        ,       0.        ],
       [   2550.        ,   21568.23632812,   79847.9921875 ,
         150827.703125  ,       0.        ,       0.        ,       0.        ],
       [      0.        ,   13260.        ,   63750.        ,
         137700.        ,  160650.        ,       0.        ,       0.        ]]

In [230]:
Dx[dmax][0]

70125.0